In [8]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import numpy as np
import pandas as pd
from collections import Counter
from sklearn import preprocessing
import scipy

In [9]:
ls data

crime_astros_finalA.csv*  crime_rockets_finalA.csv*  crime_texans_finalA.csv*
crime_astros_finalB.csv*  crime_rockets_finalB.csv*  crime_texans_finalB.csv*
crime_astros_final.csv*   crime_rockets_final.csv*   crime_texans_final.csv*
crime_dynamo_finalA.csv*  crime_ru_finalB.csv*       crime_uh_finalA.csv*
crime_dynamo_finalB.csv*  crime_ru_final.csv*        crime_uh_finalB.csv*
crime_dynamo_final.csv*   crime_ru_final.csvA.csv*   crime_uh_final.csv*


In [10]:
path = 'data/crime_texans_finalA.csv'

In [11]:
df = pd.read_csv(path)
#df.drop(['date'],axis=1,inplace=True)
df = df.set_index('date').sort_index(ascending=True)


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21843 entries, 2010-01-01 to 2017-12-31
Data columns (total 8 columns):
OffenseType     21843 non-null object
Premise         21843 non-null object
hour            21843 non-null int64
weekday         21843 non-null object
month           21843 non-null object
year            21843 non-null int64
dist_stadium    21843 non-null float64
game            21843 non-null object
dtypes: float64(1), int64(2), object(5)
memory usage: 1.5+ MB


In [13]:
df.head()

,OffenseType,Premise,hour,weekday,month,year,dist_stadium,game
date,,,,,,,,
2010-01-01,Aggravated Assault,apartment,23,Friday,Jan,2010,0.405534,No Game
2010-01-01,Aggravated Assault,apartment,23,Friday,Jan,2010,0.632841,No Game
2010-01-01,Theft,store,23,Friday,Jan,2010,0.405534,No Game
2010-01-01,Theft,store,23,Friday,Jan,2010,0.632841,No Game
2010-01-02,Theft,unk,0,Saturday,Jan,2010,0.892807,No Game


## 1 mile radius

In [14]:
one_mile =df.dist_stadium <= 1
df = df[one_mile]

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15321 entries, 2010-01-01 to 2017-12-31
Data columns (total 8 columns):
OffenseType     15321 non-null object
Premise         15321 non-null object
hour            15321 non-null int64
weekday         15321 non-null object
month           15321 non-null object
year            15321 non-null int64
dist_stadium    15321 non-null float64
game            15321 non-null object
dtypes: float64(1), int64(2), object(5)
memory usage: 1.1+ MB


# Create features
- holiday
- season


In [16]:
# import pandas as pd
# from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
# dr = pd.date_range(start='2010-01-01', end='2017-12-31')

# cal = calendar()
# holidays = cal.holidays(start=dr.min(), end=dr.max())

# df['holiday'] = df.index.isin(holidays)
# df['holiday'] = df['holiday'].astype(int)  # change to 0, or 1

# df.head()

In [17]:
def day_feature(df):
    mo = [6,7,8,9,10,11]  # morning, sunrise to 11
    af = [12,13,14,15,16]  # afternoon to fiveish
    ev = [17,18,19,20]   # evening to  sunset
    ni = [21,22,23,0,1,2,3,4,5]  # night, sunset to sunrise
    if df.hour in mo:
        val = 'Morning'
    elif df.hour in af:
        val = 'Afternoon'
    elif df.hour in ev:
        val = 'Evening'
    else:
        val = 'Night'
    return val

df['part_day'] = df.apply(day_feature,axis=1)

In [18]:
df.head()

,OffenseType,Premise,hour,weekday,month,year,dist_stadium,game,part_day
date,,,,,,,,,
2010-01-01,Aggravated Assault,apartment,23,Friday,Jan,2010,0.405534,No Game,Night
2010-01-01,Aggravated Assault,apartment,23,Friday,Jan,2010,0.632841,No Game,Night
2010-01-01,Theft,store,23,Friday,Jan,2010,0.405534,No Game,Night
2010-01-01,Theft,store,23,Friday,Jan,2010,0.632841,No Game,Night
2010-01-02,Theft,unk,0,Saturday,Jan,2010,0.892807,No Game,Night


In [19]:
def season_feature(df):
    '''
    spring (March, April, May), 
    summer (June, July, August), 
    autumn (September, October, November)
    winter (December, January, February).
    '''
    sp = ['Mar','Apr','May']   # spring
    su = ['Jun','Jul','Aug']   # summer
    au = ['Sep','Oct','Nov'] # autumn/fall
    wi = ['Dec','Jan','Feb']  # winter
    if df.month in sp:
        val = 'Spring'
    elif df.month in su:
        val = 'Summer'
    elif df.month in au:
        val = 'Autumn'
    else:
        val = 'Winter'
    return val

df['season'] = df.apply(season_feature,axis=1)

In [20]:
df.head()

,OffenseType,Premise,hour,weekday,month,year,dist_stadium,game,part_day,season
date,,,,,,,,,,
2010-01-01,Aggravated Assault,apartment,23,Friday,Jan,2010,0.405534,No Game,Night,Winter
2010-01-01,Aggravated Assault,apartment,23,Friday,Jan,2010,0.632841,No Game,Night,Winter
2010-01-01,Theft,store,23,Friday,Jan,2010,0.405534,No Game,Night,Winter
2010-01-01,Theft,store,23,Friday,Jan,2010,0.632841,No Game,Night,Winter
2010-01-02,Theft,unk,0,Saturday,Jan,2010,0.892807,No Game,Night,Winter


## Groupby date and get values

In [21]:
#df.groupby(df.index)['part_day','hour','Premise','OffenseType'].agg(lambda x: scipy.stats.mode(x, axis=None)[0][0]).head()

df['part_day_mode'] = df.groupby(df.index)['part_day'].agg(lambda x: scipy.stats.mode(x, axis=None)[0][0])
df['hour_mode'] = df.groupby(df.index)['hour'].agg(lambda x: scipy.stats.mode(x, axis=None)[0][0])
df['premise_mode'] = df.groupby(df.index)['Premise'].agg(lambda x: scipy.stats.mode(x, axis=None)[0][0])
df['offenseType_mode'] = df.groupby(df.index)['OffenseType'].agg(lambda x: scipy.stats.mode(x, axis=None)[0][0])


/home/frank/miniconda3/envs/springboard/lib/python3.6/site-packages/scipy/stats/stats.py:245: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


In [22]:
df.head()

,OffenseType,Premise,hour,weekday,month,year,dist_stadium,game,part_day,season,part_day_mode,hour_mode,premise_mode,offenseType_mode
date,,,,,,,,,,,,,,
2010-01-01,Aggravated Assault,apartment,23,Friday,Jan,2010,0.405534,No Game,Night,Winter,Night,23,apartment,Aggravated Assault
2010-01-01,Aggravated Assault,apartment,23,Friday,Jan,2010,0.632841,No Game,Night,Winter,Night,23,apartment,Aggravated Assault
2010-01-01,Theft,store,23,Friday,Jan,2010,0.405534,No Game,Night,Winter,Night,23,apartment,Aggravated Assault
2010-01-01,Theft,store,23,Friday,Jan,2010,0.632841,No Game,Night,Winter,Night,23,apartment,Aggravated Assault
2010-01-02,Theft,unk,0,Saturday,Jan,2010,0.892807,No Game,Night,Winter,Afternoon,0,hotel,Theft


In [23]:
df.apply(lambda x:sum(x.isnull()),axis=0)

OffenseType         0
Premise             0
hour                0
weekday             0
month               0
year                0
dist_stadium        0
game                0
part_day            0
season              0
part_day_mode       0
hour_mode           0
premise_mode        0
offenseType_mode    0
dtype: int64

In [24]:
df.apply(lambda x: len(x.unique()))

OffenseType           7
Premise              74
hour                 24
weekday               7
month                12
year                  8
dist_stadium        197
game                  3
part_day              4
season                4
part_day_mode         4
hour_mode            24
premise_mode         47
offenseType_mode      7
dtype: int64

In [25]:
print(list(df.columns))

['OffenseType', 'Premise', 'hour', 'weekday', 'month', 'year', 'dist_stadium', 'game', 'part_day', 'season', 'part_day_mode', 'hour_mode', 'premise_mode', 'offenseType_mode']


In [26]:
df = df[['OffenseType', 'weekday', 'month', 'year', 'dist_stadium', 'game', 'season', 'part_day_mode', 'hour_mode', 'premise_mode', 'offenseType_mode']]
df.head()

,OffenseType,weekday,month,year,dist_stadium,game,season,part_day_mode,hour_mode,premise_mode,offenseType_mode
date,,,,,,,,,,,
2010-01-01,Aggravated Assault,Friday,Jan,2010,0.405534,No Game,Winter,Night,23,apartment,Aggravated Assault
2010-01-01,Aggravated Assault,Friday,Jan,2010,0.632841,No Game,Winter,Night,23,apartment,Aggravated Assault
2010-01-01,Theft,Friday,Jan,2010,0.405534,No Game,Winter,Night,23,apartment,Aggravated Assault
2010-01-01,Theft,Friday,Jan,2010,0.632841,No Game,Winter,Night,23,apartment,Aggravated Assault
2010-01-02,Theft,Saturday,Jan,2010,0.892807,No Game,Winter,Afternoon,0,hotel,Theft


In [27]:
cdf = df.groupby(df.index).agg(
    {'OffenseType':'count',
     'weekday':'first',
     'month':'first',
     'year': 'first',
     'dist_stadium':'median',
     'season':'first',
     'part_day_mode':'first',
     'hour_mode':'first',
     'premise_mode':'first',
     'offenseType_mode':'first',
    'game':'first'})

cdf.head()

,OffenseType,weekday,month,year,dist_stadium,season,part_day_mode,hour_mode,premise_mode,offenseType_mode,game
date,,,,,,,,,,,
2010-01-01,4,Friday,Jan,2010,0.519187,Winter,Night,23,apartment,Aggravated Assault,No Game
2010-01-02,2,Saturday,Jan,2010,0.743568,Winter,Afternoon,0,hotel,Theft,No Game
2010-01-03,4,Sunday,Jan,2010,0.714016,Winter,Night,0,apartment prk,Auto Theft,Won Game
2010-01-04,1,Monday,Jan,2010,0.584825,Winter,Morning,9,apartment,Robbery,No Game
2010-01-05,2,Tuesday,Jan,2010,0.714169,Winter,Afternoon,10,prk,Auto Theft,No Game


In [28]:
df.apply(lambda x: len(x.unique()))

OffenseType           7
weekday               7
month                12
year                  8
dist_stadium        197
game                  3
season                4
part_day_mode         4
hour_mode            24
premise_mode         47
offenseType_mode      7
dtype: int64

In [29]:
# rename column
cdf.rename(columns={'OffenseType': 'crime_total','dist_stadium':'dist_stadium_meadian','offenseType_mode':'offense_mode'}, inplace=True)

In [30]:
cdf.head()

,crime_total,weekday,month,year,dist_stadium_meadian,season,part_day_mode,hour_mode,premise_mode,offense_mode,game
date,,,,,,,,,,,
2010-01-01,4,Friday,Jan,2010,0.519187,Winter,Night,23,apartment,Aggravated Assault,No Game
2010-01-02,2,Saturday,Jan,2010,0.743568,Winter,Afternoon,0,hotel,Theft,No Game
2010-01-03,4,Sunday,Jan,2010,0.714016,Winter,Night,0,apartment prk,Auto Theft,Won Game
2010-01-04,1,Monday,Jan,2010,0.584825,Winter,Morning,9,apartment,Robbery,No Game
2010-01-05,2,Tuesday,Jan,2010,0.714169,Winter,Afternoon,10,prk,Auto Theft,No Game


In [31]:
path

'data/crime_texans_finalA.csv'

In [32]:
cdf.to_csv('data/crime_texans_finalB.csv')